In [14]:
from pathlib import Path
import numpy as np 
import xml.etree.ElementTree as ET
from sklearn.svm import SVC
from skimage.io import imread

from pillclassification.feature_extraction import feature_extraction

images_dir = Path('utils/Dataset/merge')
filenames = [x for x in images_dir.iterdir() if x.suffix != '.xml']

samples_num = len(filenames)samples_num = len(filenames)
feature_number = 10

In [11]:
# calculating labels 
try:
    tree = ET.parse(images_dir / 'images.xml')
except ET.ParseError:
    print('Parse error on {}'.format(images_dir / 'images.xml'))
    exit(-1)

se = list(tree.getroot())[0]

labels_set = set()
for e in list(se):
    labels_set.add(e.find('NDC9').text)

labels = sorted(list(labels_set))
class_num = len(labels)


In [17]:
# extracting features
x_data = np.zeros((samples_num, feature_number))
y_data = np.zeros(samples_num)

for i, f in enumerate(filenames):
    # features 
    hu, rgb_val = feature_extraction(imread(str(f)))
    x_data[i, :6] = hu
    x_data[i, 7:] = rgb_val

    for e in list(se):
        if e.find('File').find('Name').text == f:
            y_data[i] = labels.index(e.find('NDC9'))
        else:
            y_data[i] = -1

print(y_data)

In [13]:
# SVC training
kernel = 'linear'
max_iteration = 10

models = []
for i in range(class_num):
    models.append(SVC(kernel=kernel, max_iter=max_iteration, probability=True))

for i in range(len(labels)):
    models[i].fit(x_data, y_data == i) # training 


ValueError: The number of classes has to be greater than one; got 1 class